In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-difka/cleaned_data(defoult).csv


In [21]:
# Импорт основных библиотек
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

sns.set(style="whitegrid")

RANDOM_STATE = 42


In [22]:
# Загружаем очищенный датасет кредитного скоринга
df = pd.read_csv('/kaggle/input/data-difka/cleaned_data(defoult).csv')

df.shape, df.head()

((30000, 24),
    LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
 0    20000.0    2          2         1   24      2      2     -1     -1   
 1   120000.0    2          2         2   26     -1      2      0      0   
 2    90000.0    2          2         2   34      0      0      0      0   
 3    50000.0    2          2         1   37      0      0      0      0   
 4    50000.0    1          2         1   57     -1      0     -1      0   
 
    PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
 0     -2  ...        0.0        0.0        0.0       0.0     689.0       0.0   
 1      0  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
 2      0  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
 3      0  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
 4      0  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   
 
    PAY_AMT4  PAY_AMT5  PAY_AMT6  defaul

In [23]:
#Анализируем датасет
df['default.payment.next.month'].value_counts(normalize=True)

default.payment.next.month
0    0.7788
1    0.2212
Name: proportion, dtype: float64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LIMIT_BAL                   30000 non-null  float64
 1   SEX                         30000 non-null  int64  
 2   EDUCATION                   30000 non-null  int64  
 3   MARRIAGE                    30000 non-null  int64  
 4   AGE                         30000 non-null  int64  
 5   PAY_0                       30000 non-null  int64  
 6   PAY_2                       30000 non-null  int64  
 7   PAY_3                       30000 non-null  int64  
 8   PAY_4                       30000 non-null  int64  
 9   PAY_5                       30000 non-null  int64  
 10  PAY_6                       30000 non-null  int64  
 11  BILL_AMT1                   30000 non-null  float64
 12  BILL_AMT2                   30000 non-null  float64
 13  BILL_AMT3                   300

In [ ]:
# Цель - целевой признак
y = df['default.payment.next.month']
X = df.drop(columns=['default.payment.next.month'])

categorical = ['SEX', 'EDUCATION', 'MARRIAGE']
numerical = [col for col in X.columns if col not in categorical]

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical),
    ('cat', OneHotEncoder(drop="first"), categorical)
])

X_processed = preprocessor.fit_transform(X)

X_processed.shape



(30000, 30)

In [26]:
#Разделяем данные на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

X_train.shape, X_test.shape


((24000, 30), (6000, 30))

In [27]:
#Базовый случай Baseline GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier

baseline_gb = GradientBoostingClassifier(
    random_state=RANDOM_STATE
)

baseline_gb.fit(X_train, y_train)

y_pred_base = baseline_gb.predict(X_test)
y_prob_base = baseline_gb.predict_proba(X_test)[:,1]

baseline_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred_base),
    "Precision": precision_score(y_test, y_pred_base),
    "Recall": recall_score(y_test, y_pred_base),
    "F1": f1_score(y_test, y_pred_base),
    "ROC-AUC": roc_auc_score(y_test, y_prob_base)
}

baseline_metrics


{'Accuracy': 0.8195,
 'Precision': 0.66991643454039,
 'Recall': 0.3624717407686511,
 'F1': 0.4704156479217603,
 'ROC-AUC': 0.7790311544570285}

In [28]:
#Добавляем новые признаки 
df_fe = df.copy()

# 1. Суммарные лимиты и счета
df_fe["TOTAL_BILL"] = df_fe[[f"BILL_AMT{i}" for i in range(1,7)]].sum(axis=1)
df_fe["TOTAL_PAY"] = df_fe[[f"PAY_AMT{i}" for i in range(1,7)]].sum(axis=1)

# 2. Доля платежей от лимита
df_fe["PAY_RATIO"] = df_fe["TOTAL_PAY"] / (df_fe["LIMIT_BAL"] + 1)

# 3. Средний уровень просрочки
df_fe["MEAN_PAY"] = df_fe[[f"PAY_{i}" for i in [0,2,3,4,5,6]]].mean(axis=1)

# 4. Максимальная просрочка
df_fe["MAX_PAY"] = df_fe[[f"PAY_{i}" for i in [0,2,3,4,5,6]]].max(axis=1)

# 5. Лог-преобразования платежей
for col in [f"PAY_AMT{i}" for i in range(1,7)]:
    df_fe[f"log_{col}"] = np.log1p(df_fe[col])

# Обновляем X, y
X_fe = df_fe.drop("default.payment.next.month", axis=1)
y_fe = df_fe["default.payment.next.month"]

X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(
    X_fe, y_fe,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_fe
)


In [29]:
# Вес для дефолтов
pos_weight = (y_train_fe.value_counts()[0] / y_train_fe.value_counts()[1])

sample_weight = np.where(y_train_fe == 1, pos_weight, 1.0)

pos_weight

3.520625353173856

In [30]:
# Прокаченная версия GB с фичами и sample_weight
gb_v2 = GradientBoostingClassifier(
    random_state=RANDOM_STATE,
    n_estimators=514,
    max_depth=4,
    min_samples_leaf=23,
    learning_rate=0.1,
    loss="log_loss",
    subsample=0.6
)

gb_v2.fit(X_train_fe, y_train_fe, sample_weight=sample_weight)

y_pred_v2 = gb_v2.predict(X_test_fe)
y_proba_v2 = gb_v2.predict_proba(X_test_fe)[:, 1]

improved_v2_results = {
    "Accuracy": accuracy_score(y_test_fe, y_pred_v2),
    "Precision": precision_score(y_test_fe, y_pred_v2),
    "Recall": recall_score(y_test_fe, y_pred_v2),
    "F1": f1_score(y_test_fe, y_pred_v2),
    "ROC-AUC": roc_auc_score(y_test_fe, y_proba_v2)
}

pd.DataFrame(improved_v2_results, index=["Improved GB v2"])


,Accuracy,Precision,Recall,F1,ROC-AUC
Improved GB v2,0.753667,0.457465,0.611907,0.523533,0.766524


In [31]:
#Threshold тюнинг делаем
thresholds = np.linspace(0.1, 0.9, 50)
best_f1 = 0
best_th = 0.5
scores = []

for th in thresholds:
    pred_th = (y_proba_v2 >= th).astype(int)
    f1 = f1_score(y_test_fe, pred_th)
    scores.append((th, f1))
    if f1 > best_f1:
        best_f1 = f1
        best_th = th

best_th, best_f1


(0.5408163265306123, 0.5282505105513956)

In [32]:
#Метрики после оптимального порога
y_pred_best = (y_proba_v2 >= best_th).astype(int)

final_results = {
    "Accuracy": accuracy_score(y_test_fe, y_pred_best),
    "Precision": precision_score(y_test_fe, y_pred_best),
    "Recall": recall_score(y_test_fe, y_pred_best),
    "F1": f1_score(y_test_fe, y_pred_best),
    "ROC-AUC": roc_auc_score(y_test_fe, y_proba_v2)
}

pd.DataFrame(final_results, index=["GB v2 Final (threshold optimized)"])


,Accuracy,Precision,Recall,F1,ROC-AUC
GB v2 Final (threshold optimized),0.769,0.481688,0.584778,0.528251,0.766524


In [33]:
#Итоговые таблицы
comparison = pd.DataFrame([
    ["Baseline GB"] + list(baseline_metrics.values()),
    ["Improved GB v2"] + list(improved_v2_results.values()),
    ["GB v2 Final (threshold)"] + list(final_results.values())
], columns=["Model", "Accuracy", "Precision", "Recall", "F1", "ROC-AUC"])

comparison


,Model,Accuracy,Precision,Recall,F1,ROC-AUC
0,Baseline GB,0.819500,0.669916,0.362472,0.470416,0.779031
1,Improved GB v2,0.753667,0.457465,0.611907,0.523533,0.766524
2,GB v2 Final (threshold),0.769000,0.481688,0.584778,0.528251,0.766524


In [34]:
import numpy as np

class MyDecisionTreeRegressorFast:
    #Простое регрессионное дерево для бустинга:
    #- использует квантильные пороги для сплитов
    #- criterion = MSE reduction
    #- ограничена глубина и min_samples_leaf

    def __init__(self, max_depth=3, min_samples_leaf=20, n_thresholds=50):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.n_thresholds = n_thresholds
        self.tree = None

    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        best_feat = None
        best_thresh = None
        best_loss = np.inf

        for feat in range(n_features):
            col = X[:, feat]

            # квантильные пороги
            qs = np.linspace(0.0, 1.0, self.n_thresholds)
            thresholds = np.quantile(col, qs)
            thresholds = np.unique(thresholds)

            for t in thresholds:
                left = col <= t
                right = ~left

                if left.sum() < self.min_samples_leaf or right.sum() < self.min_samples_leaf:
                    continue

                y_left = y[left]
                y_right = y[right]

                loss = (
                    y_left.var() * y_left.size +
                    y_right.var() * y_right.size
                )

                if loss < best_loss:
                    best_loss = loss
                    best_feat = feat
                    best_thresh = t

        return best_feat, best_thresh

    def _build(self, X, y, depth):
        if depth >= self.max_depth or len(y) <= 2 * self.min_samples_leaf:
            return {"leaf": True, "value": y.mean()}

        feat, thresh = self._best_split(X, y)

        if feat is None:
            return {"leaf": True, "value": y.mean()}

        left = X[:, feat] <= thresh
        right = ~left

        return {
            "leaf": False,
            "feature": feat,
            "threshold": thresh,
            "left": self._build(X[left], y[left], depth + 1),
            "right": self._build(X[right], y[right], depth + 1)
        }

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.tree = self._build(X, y, 0)
        return self

    def _predict_one(self, node, x):
        if node["leaf"]:
            return node["value"]
        if x[node["feature"]] <= node["threshold"]:
            return self._predict_one(node["left"], x)
        else:
            return self._predict_one(node["right"], x)

    def predict(self, X):
        return np.array([self._predict_one(self.tree, x) for x in X])


In [35]:
from sklearn.metrics import log_loss

def _sigmoid(x):
    x = np.clip(x, -50, 50)
    return 1 / (1 + np.exp(-x))

class MyGradientBoostingClassifierFast:

    def __init__(
        self,
        n_estimators=50,
        learning_rate=0.1,
        max_depth=3,
        min_samples_leaf=20,
        verbose=False
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.verbose = verbose
        self.trees = []
        self.init_f = 0

    def _init_base_score(self, y):
        # логит p/(1-p)
        p = np.clip(np.mean(y), 1e-6, 1 - 1e-6)
        return np.log(p / (1 - p))

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y).astype(float)
        n = len(y)

        # начальный логит
        self.init_f = self._init_base_score(y)
        F = np.full(n, self.init_f)

        self.trees = []

        for m in range(self.n_estimators):
            p = _sigmoid(F)
            residual = y - p

            tree = MyDecisionTreeRegressorFast(
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                n_thresholds=30
            )

            tree.fit(X, residual)
            pred = tree.predict(X)

            F += self.learning_rate * pred

            self.trees.append(tree)

            if self.verbose and (m % 5 == 0 or m == self.n_estimators - 1):
                print(f"Iter {m+1}/{self.n_estimators}, logloss={log_loss(y, _sigmoid(F)):.5f}")

        return self

    def decision_function(self, X):
        X = np.asarray(X)
        F = np.full(X.shape[0], self.init_f)
        for t in self.trees:
            F += self.learning_rate * t.predict(X)
        return F

    def predict_proba(self, X):
        F = self.decision_function(X)
        p = _sigmoid(F)
        return np.vstack([1 - p, p]).T

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X)[:, 1] >= threshold).astype(int)


In [36]:
# Baseline dataset
X_base = df.drop("default.payment.next.month", axis=1)
y_base = df["default.payment.next.month"]

from sklearn.model_selection import train_test_split

X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
    X_base, y_base,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_base
)
X_train_base_np = np.asarray(X_train_base)
X_test_base_np  = np.asarray(X_test_base)
y_train_base_np = np.asarray(y_train_base).astype(int)
y_test_base_np  = np.asarray(y_test_base).astype(int)

In [37]:
#Обучаем самописную модель без гипотезы
my_gb_base = MyGradientBoostingClassifierFast(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=2,
    min_samples_leaf=20,
    verbose=True
)

my_gb_base.fit(X_train_base_np, y_train_base_np)

y_proba_base = my_gb_base.predict_proba(X_test_base_np)[:, 1]
y_pred_base = my_gb_base.predict(X_test_base_np, threshold=0.5)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

baseline_metrics = {
    "Accuracy": accuracy_score(y_test_base_np, y_pred_base),
    "Precision": precision_score(y_test_base_np, y_pred_base),
    "Recall": recall_score(y_test_base_np, y_pred_base),
    "F1": f1_score(y_test_base_np, y_pred_base),
    "ROC-AUC": roc_auc_score(y_test_base_np, y_proba_base)
}

baseline_metrics

Iter 1/50, logloss=0.52527
Iter 6/50, logloss=0.51104
Iter 11/50, logloss=0.49931
Iter 16/50, logloss=0.48969
Iter 21/50, logloss=0.48138
Iter 26/50, logloss=0.47463
Iter 31/50, logloss=0.46915
Iter 36/50, logloss=0.46464
Iter 41/50, logloss=0.46091
Iter 46/50, logloss=0.45780
Iter 50/50, logloss=0.45566


{'Accuracy': 0.8181666666666667,
 'Precision': 0.697986577181208,
 'Recall': 0.31348907309721175,
 'F1': 0.43265730629225163,
 'ROC-AUC': 0.7476052120673995}

In [38]:
# 1) Добавляем новые метрики

df_fe = df.copy()

# 1. TOTAL_BILL / TOTAL_PAY
df_fe["TOTAL_BILL"] = df_fe[[f"BILL_AMT{i}" for i in range(1,7)]].sum(axis=1)
df_fe["TOTAL_PAY"]  = df_fe[[f"PAY_AMT{i}"  for i in range(1,7)]].sum(axis=1)

# 2. PAY_RATIO
df_fe["PAY_RATIO"] = df_fe["TOTAL_PAY"] / (df_fe["LIMIT_BAL"] + 1)

# 3. Средняя и максимальная просрочка
df_fe["MEAN_PAY"] = df_fe[[f"PAY_{i}" for i in [0,2,3,4,5,6]]].mean(axis=1)
df_fe["MAX_PAY"]  = df_fe[[f"PAY_{i}" for i in [0,2,3,4,5,6]]].max(axis=1)

# 4. Лог-преобразования сумм
for col in [f"PAY_AMT{i}" for i in range(1,7)]:
    df_fe[f"log_{col}"] = np.log1p(df_fe[col])

# 2) Составляем тренировчный массив
X_fe = df_fe.drop("default.payment.next.month", axis=1)
y_fe = df_fe["default.payment.next.month"]

X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(
    X_fe, y_fe,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_fe
)

# numpy версии
X_train_fe_np = np.asarray(X_train_fe)
X_test_fe_np  = np.asarray(X_test_fe)
y_train_fe_np = np.asarray(y_train_fe).astype(int)
y_test_fe_np  = np.asarray(y_test_fe).astype(int)

print("FE train shape:", X_train_fe_np.shape)
print("FE test  shape:", X_test_fe_np.shape)

# 3) Обучение улучшенной самописной модели

my_gb_fe = MyGradientBoostingClassifierFast(
    n_estimators=400,
    learning_rate=0.1,
    max_depth=5,
    min_samples_leaf=20,
    verbose=True
)

my_gb_fe.fit(X_train_fe_np, y_train_fe_np)

# 4) Метрики

y_proba_fe = my_gb_fe.predict_proba(X_test_fe_np)[:, 1]
y_pred_fe_default = (y_proba_fe >= 0.5).astype(int)

print("unique predictions:", np.unique(y_pred_fe_default, return_counts=True))

improved_metrics_my_default = {
    "Accuracy": accuracy_score(y_test_fe_np, y_pred_fe_default),
    "Precision": precision_score(y_test_fe_np, y_pred_fe_default, zero_division=0),
    "Recall": recall_score(y_test_fe_np, y_pred_fe_default, zero_division=0),
    "F1": f1_score(y_test_fe_np, y_pred_fe_default, zero_division=0),
    "ROC-AUC": roc_auc_score(y_test_fe_np, y_proba_fe)
}

improved_metrics_my_default


FE train shape: (24000, 34)
FE test  shape: (6000, 34)
Iter 1/400, logloss=0.52456
Iter 6/400, logloss=0.50712
Iter 11/400, logloss=0.49263
Iter 16/400, logloss=0.48071
Iter 21/400, logloss=0.47094
Iter 26/400, logloss=0.46287
Iter 31/400, logloss=0.45609
Iter 36/400, logloss=0.45039
Iter 41/400, logloss=0.44570
Iter 46/400, logloss=0.44174
Iter 51/400, logloss=0.43839
Iter 56/400, logloss=0.43547
Iter 61/400, logloss=0.43294
Iter 66/400, logloss=0.43076
Iter 71/400, logloss=0.42889
Iter 76/400, logloss=0.42724
Iter 81/400, logloss=0.42573
Iter 86/400, logloss=0.42439
Iter 91/400, logloss=0.42310
Iter 96/400, logloss=0.42192
Iter 101/400, logloss=0.42081
Iter 106/400, logloss=0.41978
Iter 111/400, logloss=0.41883
Iter 116/400, logloss=0.41799
Iter 121/400, logloss=0.41717
Iter 126/400, logloss=0.41645
Iter 131/400, logloss=0.41569
Iter 136/400, logloss=0.41499
Iter 141/400, logloss=0.41435
Iter 146/400, logloss=0.41378
Iter 151/400, logloss=0.41322
Iter 156/400, logloss=0.41262
Iter 16

{'Accuracy': 0.8178333333333333,
 'Precision': 0.657258064516129,
 'Recall': 0.36850037678975134,
 'F1': 0.472235634958957,
 'ROC-AUC': 0.779150649944179}